In [ ]:
!pip install -q tensorflow

In [ ]:
!pip install -q numpy

In [ ]:
!pip install -q matplotlib

In [ ]:
!pip install -q keras

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from keras.utils import load_img, img_to_array

In [ ]:
def load_and_preprocess_data():
    """
    Loads and preprocesses the image data for training and validation.
    """
    source_folder = "hateful_memes/img"
    train_folder = "train_data"
    validation_folder = "validation_data"

    train_dir = os.path.join(source_folder, train_folder)
    validation_dir = os.path.join(source_folder, validation_folder)

    train_hateful_memes_dir = os.path.join(train_dir, 'hateful_memes')
    train_not_hateful_memes_dir = os.path.join(train_dir, 'not_hateful_memes')
    validation_hateful_memes_dir = os.path.join(validation_dir, 'hateful_memes')
    validation_not_hateful_memes_dir = os.path.join(validation_dir, 'not_hateful_memes')

    print('total training hateful images:', len(os.listdir(train_hateful_memes_dir)))
    print('total training not hateful images:', len(os.listdir(train_not_hateful_memes_dir)))
    print('total validation hateful images:', len(os.listdir(validation_hateful_memes_dir)))
    print('total validation not hateful images:', len(os.listdir(validation_not_hateful_memes_dir)))

    train_datagen = ImageDataGenerator(
        rescale=1./255.,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    validation_datagen = ImageDataGenerator(rescale=1.0/255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        classes=['hateful_memes', 'not_hateful_memes'],
        target_size=(224, 224),
        batch_size=10,
        class_mode='binary'
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        classes=['hateful_memes', 'not_hateful_memes'],
        target_size=(224, 224),
        batch_size=10,
        class_mode='binary',
        shuffle=False
    )
    return train_generator, validation_generator

In [ ]:
def build_and_compile_model(base_model):
    """
    Builds and compiles the model.
    """
    for layer in base_model.layers:
        layer.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.models.Model(base_model.input, x)
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
                  loss='binary_crossentropy',
                  metrics=['acc'])
    return model

In [ ]:
def train_model(model, train_generator, validation_generator):
    """
    Trains the model.
    """
    vgghist = model.fit(
        train_generator,
        steps_per_epoch=480,
        epochs=10,
        verbose=1,
        validation_data=validation_generator,
        validation_steps=120
    )
    return vgghist

In [ ]:
def evaluate_model(model, validation_generator):
    """
    Evaluates the model on validation data.
    """
    val_loss, val_acc = model.evaluate(validation_generator)
    print("Validation Loss:", val_loss)
    print("Validation Accuracy:", val_acc)

In [ ]:
def save_model(model, filename):
    """
    Saves the model.
    """
    model.save(filename)

In [ ]:
def predict_images(model, image_directory):
    """
    Predicts classes of images in a directory.
    """
    image_files = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

    for fn in image_files:
        path = os.path.join(image_directory, fn)
        img = load_img(path, target_size=(224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        images = np.vstack([x])
        classes = model.predict(images, batch_size=1)
        if classes[0] < 0.5:
            print(fn + " is a Hateful Meme")
        else:
            print(fn + " is a Not Hateful Meme")

def plot_training_history(vgghist):
    """
    Plots training and validation accuracy and loss.
    """
    plt.plot(vgghist.history['acc'])
    plt.plot(vgghist.history['val_acc'])
    plt.title('Model accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    plt.plot(vgghist.history['loss'])
    plt.plot(vgghist.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

# Load and preprocess data
train_generator, validation_generator = load_and_preprocess_data()

# Build and compile the model
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
model = build_and_compile_model(base_model)
model.summary()

# Train the model
vgghist = train_model(model, train_generator, validation_generator)

# Evaluate the model
evaluate_model(model, validation_generator)

# Save the model
save_model(model, 'Classification_VGG16.keras')

# Predict classes of images in a directory
local_image_directory = "C:\\Users\\chira\\Documents\\Projts\\Analysis Hateful Memes\\test"
predict_images(model, local_image_directory)

# Plot training history
plot_training_history(vgghist)